In [ ]:
import os
import pickle
import gc
from modules.logging.format_utils import format_measures
from modules.collecting.results_collector import DataFrameCollector
from modules.logging.logger import DefaultLogger
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode
from modules.algorithms.nn.OSLPP_NN_UTILS import train

In [ ]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [ ]:
config

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1'

In [ ]:
from modules.deep.DEEPNN import get_simple_deep_nn, fix_batch_normalization_layers


def get_model(params, n_classes):
    m = get_simple_deep_nn(n_classes).cuda().train()
    m.apply(fix_batch_normalization_layers)
    return m

In [ ]:
from modules.algorithms.base.OSLPP import Params, do_l2_normalization, do_pca
from modules.loaders.common import set_seed

results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
for (source, target), (common, tgt_private) in config.items():
    for epochs in [50]:
        for lr in [1e-4]:
            for n_r in [0.15, 0.25]:
                seed = 0
                set_seed(seed)
                params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                          dataset='DomainNet_DCC', source=source, target=target,
                          num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                metrics = train(params, lr, epochs, 1, select_reject_mode, seed, common, tgt_private, logger, get_model=get_model, is_images=True)
                results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs})
                gc.collect()
                results.get_result().to_csv(f'all_results/results_deep/dnn_small_confidence_deep_{source}_{target}.csv', header=True, index=False)

In [ ]:
results.to_csv('results/dnn_small_confidence_deep.csv', header=True, index=False)